In [1]:
import os
import ast
import torch
import subprocess
from transformers import AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from torch.utils.data import Dataset
import gc
from transformers import Qwen2ForCausalLM

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Параметры 
LOCAL_MODEL_DIR = "Qwen2.5-Coder-3B"
MAX_LEN = 512
EPOCHS = 2
BATCH_SIZE = 2
LR = 5e-6
OUTPUT_DIR = "./ast_trained_github"
LIMIT = 10000

REPOS = [
    "https://github.com/psf/requests.git",
    "https://github.com/pallets/flask.git", 
    "https://github.com/pandas-dev/pandas.git",
    "https://github.com/numpy/numpy.git",
    "https://github.com/scipy/scipy.git",
    "https://github.com/scikit-learn/scikit-learn.git",
    "https://github.com/matplotlib/matplotlib.git",
    "https://github.com/plotly/plotly.py.git",
    "https://github.com/pytorch/pytorch.git",
    "https://github.com/tensorflow/tensorflow.git"
]

CLONE_DIR = "./github_repos"
os.makedirs(CLONE_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Проверяем наличие локальной модели
if not os.path.exists(LOCAL_MODEL_DIR):
    print(f"Локальная модель не найдена: {LOCAL_MODEL_DIR}")
    exit(1)
else:
    print(f"Локальная модель найдена: {LOCAL_MODEL_DIR}")

# Клонирование репозиториев 
print("📁 Клонируем репозитории...")
for repo in REPOS:
    name = repo.split("/")[-1].replace(".git", "")
    dest = os.path.join(CLONE_DIR, name)
    if not os.path.exists(dest):
        try:
            subprocess.run(["git", "clone", "--depth", "1", repo, dest],
                           capture_output=True, text=True, timeout=300)
            print(f"{name} клонирован")
        except Exception as e:
            print(f"Ошибка клонирования {repo}: {e}")

# Извлечение AST 
def extract_ast_sequence(code: str) -> str:
    """
    Преобразует Python-код в линейное AST-представление
    """
    try:
        tree = ast.parse(code)
        return ast.dump(tree, annotate_fields=True, include_attributes=False)
    except SyntaxError:
        return ""
    except Exception as e:
        print(f"Ошибка AST: {e}")
        return ""

def visualize_ast_debug(code: str, filename: str):
    """
    Отладочная печать AST для примера
    """
    try:
        tree = ast.parse(code)
        print(f"\n AST для {filename}:")
        print(ast.dump(tree, indent=2, annotate_fields=True, include_attributes=False)[:500])
    except Exception as e:
        print(f"Ошибка визуализации AST: {e}")

# Сбор Python файлов
texts = []
count = 0

print("Сбор Python файлов и построение AST...")
for root, dirs, files in os.walk(CLONE_DIR):
    for file in files:
        if file.endswith(".py") and not file.startswith("test_"):
            path = os.path.join(root, file)
            try:
                with open(path, "r", encoding="utf-8") as f:
                    code = f.read()

                seq = extract_ast_sequence(code)
                if seq and len(seq) > 20:
                    texts.append({"content": seq})
                    count += 1

                    if count <= 3:
                        visualize_ast_debug(code, file)
                        print(f"Пример AST: {seq[:100]}...")

                if count % 50 == 0:
                    print(f"Обработано {count} файлов")

            except Exception as e:
                print(f"Ошибка обработки {file}: {e}")
                continue

        if count >= LIMIT:
            break
    if count >= LIMIT:
        break

print(f"Собрано {len(texts)} примеров AST")

if len(texts) == 0:
    print("Нет данных, создаём тестовые примеры...")
    samples = [
        "def add(a,b): return a+b",
        "class C: pass",
        "x = [i*i for i in range(5)]"
    ]
    for s in samples:
        seq = extract_ast_sequence(s)
        if seq:
            texts.append({"content": seq})
            print(f"Тестовый пример: {seq[:80]}...")
    print(f"Создано {len(texts)} AST-примеров")

# Токенизация 
print("Загружаем токенизатор...")
tokenizer = AutoTokenizer.from_pretrained(
    LOCAL_MODEL_DIR,
    trust_remote_code=True,
    local_files_only=True
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Токенизатор готов")

print("Токенизация данных...")
tokenized_data = []
for i, t in enumerate(texts):
    tok = tokenizer(
        t["content"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
        return_tensors="pt"
    )
    tok["labels"] = tok["input_ids"].clone()
    tokenized_data.append(tok)
    if (i + 1) % 100 == 0:
        print(f"Токенизировано {i + 1}")

print(f"Всего токенизировано {len(tokenized_data)} примеров")

# Dataset
class ASTDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        d = self.data[idx]
        return {
            'input_ids': d['input_ids'].squeeze(0),
            'attention_mask': d['attention_mask'].squeeze(0),
            'labels': d['labels'].squeeze(0)
        }

dataset = ASTDataset(tokenized_data)
print("AST Dataset создан")

# Загрузка модели 
print("Загружаем модель...")
try:
    model = Qwen2ForCausalLM.from_pretrained(
        LOCAL_MODEL_DIR,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        local_files_only=True
    )
    print("Модель загружена (float16)")
except Exception as e:
    print(f"Ошибка float16: {e}")
    model = Qwen2ForCausalLM.from_pretrained(
        LOCAL_MODEL_DIR,
        trust_remote_code=True,
        device_map="auto",
        local_files_only=True
    )
    print("Модель загружена без float16")

# Настройка LoRA 
print("Настройка LoRA...")
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("LoRA активна")

# Обучение 
print("Настройка Trainer...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=4,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    warmup_steps=50,
    logging_steps=10,
    save_steps=200,
    save_total_limit=1,
    fp16=torch.cuda.is_available() and model.dtype == torch.float16,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    report_to="none",
    disable_tqdm=False,
)

def collate(batch):
    return {
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
        'labels': torch.stack([x['labels'] for x in batch])
    }

trainer = Trainer(model=model, args=training_args, train_dataset=dataset, data_collator=collate)

print("Начинаем обучение модели на AST...")
try:
    trainer.train()
    print("Обучение завершено")
except Exception as e:
    print(f"Ошибка обучения: {e}")

# ---------------- 10. Сохранение ----------------
print("Сохраняем модель...")
try:
    trainer.save_model()
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"💾 Модель сохранена в {OUTPUT_DIR}")

    print(" Тест генерации...")
    test_input = "Module(body=[FunctionDef(name='foo', args=arguments("
    tokens = tokenizer(test_input, return_tensors="pt").to(model.device)
    out = model.generate(**tokens, max_length=60, temperature=0.7)
    print("📝", tokenizer.decode(out[0], skip_special_tokens=True))
except Exception as e:
    print(f" Ошибка сохранения: {e}")

# Очистка 
del model, trainer, dataset, tokenized_data
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Обучение AST модели завершено!")


✅ Локальная модель найдена: Qwen2.5-Coder-3B
📁 Клонируем репозитории...
📁 Сбор Python файлов и построение AST...

🔍 AST для _array_api_info.py:
Module(
  body=[
    Expr(
      value=Constant(value='\nArray API Inspection namespace\n\nThis is the namespace for inspection functions as defined by the array API\nstandard. See\nhttps://data-apis.org/array-api/latest/API_specification/inspection.html for\nmore details.\n\n')),
    ImportFrom(
      module='numpy._core',
      names=[
        alias(name='bool'),
        alias(name='complex64'),
        alias(name='complex128'),
        alias(name='dtype'),
        alias(name='float32'),
     
Пример AST: Module(body=[Expr(value=Constant(value='\nArray API Inspection namespace\n\nThis is the namespace fo...

🔍 AST для _configtool.py:
Module(
  body=[
    Import(
      names=[
        alias(name='argparse')]),
    Import(
      names=[
        alias(name='sys')]),
    ImportFrom(
      module='pathlib',
      names=[
        alias(name='Path')

<unknown>:17: SyntaxWarning: invalid escape sequence '\w'


✅ Обработано 2200 файлов
✅ Обработано 2250 файлов
✅ Обработано 2300 файлов
✅ Обработано 2350 файлов
✅ Обработано 2400 файлов
✅ Обработано 2450 файлов
✅ Обработано 2500 файлов
✅ Обработано 2550 файлов
✅ Обработано 2600 файлов
✅ Обработано 2650 файлов
✅ Обработано 2700 файлов
✅ Обработано 2750 файлов
✅ Обработано 2800 файлов
✅ Обработано 2850 файлов
✅ Обработано 2900 файлов
✅ Обработано 2950 файлов
✅ Обработано 3000 файлов
✅ Обработано 3050 файлов
✅ Обработано 3100 файлов
✅ Обработано 3150 файлов
✅ Обработано 3200 файлов
✅ Обработано 3250 файлов
✅ Обработано 3300 файлов
✅ Обработано 3350 файлов
✅ Обработано 3400 файлов
✅ Обработано 3450 файлов
✅ Обработано 3500 файлов
✅ Обработано 3550 файлов
✅ Обработано 3600 файлов
✅ Обработано 3650 файлов
✅ Обработано 3700 файлов
✅ Обработано 3750 файлов
✅ Обработано 3800 файлов
✅ Обработано 3850 файлов
✅ Обработано 3900 файлов
✅ Обработано 3950 файлов
✅ Обработано 4000 файлов
✅ Обработано 4050 файлов
✅ Обработано 4100 файлов
✅ Обработано 4150 файлов


<unknown>:95: SyntaxWarning: invalid escape sequence '\s'


✅ Обработано 7250 файлов
✅ Обработано 7300 файлов
✅ Обработано 7350 файлов
✅ Обработано 7400 файлов
✅ Обработано 7450 файлов
✅ Обработано 7500 файлов
✅ Обработано 7550 файлов
✅ Обработано 7600 файлов
✅ Обработано 7650 файлов
✅ Обработано 7700 файлов
✅ Обработано 7750 файлов
✅ Обработано 7800 файлов
✅ Обработано 7850 файлов
✅ Обработано 7900 файлов
✅ Обработано 7950 файлов
✅ Обработано 8000 файлов
✅ Обработано 8050 файлов
✅ Обработано 8100 файлов
✅ Обработано 8150 файлов
✅ Обработано 8200 файлов
✅ Обработано 8250 файлов
✅ Обработано 8300 файлов
✅ Обработано 8350 файлов
✅ Обработано 8400 файлов
✅ Обработано 8450 файлов
✅ Обработано 8500 файлов
✅ Обработано 8550 файлов
✅ Обработано 8600 файлов
✅ Обработано 8650 файлов
✅ Обработано 8700 файлов
✅ Обработано 8750 файлов


...

trainable params: 3,686,400 || all params: 3,089,625,088 || trainable%: 0.1193
✅ LoRA активна
🚀 Настройка Trainer...
🚀 Начинаем обучение модели на AST...


Step,Training Loss
10,2.439700
20,1.855300
30,1.623000
40,1.960400
50,1.529100
60,2.058900
70,1.533400
80,1.786600
90,1.190700
100,1.276400


✅ Обучение завершено
💾 Сохраняем модель...
💾 Модель сохранена в ./ast_trained_github
🧪 Тест генерации...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


📝 Module(body=[FunctionDef(name='foo', args=arguments(posonlyargs=[], args=[arg(arg='x')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Expr(value=Constant(value='foo')), Return(value=Name(id='x', ctx=Load()))], decorator_list=[], type_params=[]), FunctionDef(name='bar', args=arguments(posonlyargs=[], args=[arg(arg='x')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Expr(value=Constant(value='bar')), Return(value=Name(id='x', ctx=Load()))], decorator_list=[], type_params=[]), FunctionDef(name='baz', args=arguments(posonlyargs=[], args=[arg(arg='x')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Expr(value=Constant(value='baz')), Return(value=Name(id='x', ctx=Load()))], decorator_list=[], type_params=[]), FunctionDef(name='qux', args=arguments(posonlyargs=[], args=[arg(arg='x')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Expr(value=Constant(value='qux')), Return(value=Name(id='x', ctx=Load()))], decorator_list=[], type_params=[]), FunctionDef(name='quux', args

In [2]:
import zipfile
import os

def zip_directory(directory_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, directory_path)
                zipf.write(file_path, arcname)

# Использование
zip_directory('ast_trained_github', 'ast_trained_github.zip')